In [ ]:
import json
import argparse
import os
import sys
from utils import print_response
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams



In [ ]:
# parser for .py version

# parser = argparse.ArgumentParser()

# parser.add_argument('--paper_name',type=str)

# parser.add_argument('--model_name',type=str, default="deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct") 
# parser.add_argument('--tp_size',type=int, default=2)
# parser.add_argument('--temperature',type=float, default=1.0)
# parser.add_argument('--max_model_len',type=int, default=128000)

# parser.add_argument('--paper_format',type=str, default="JSON", choices=["JSON", "LaTeX"])
# parser.add_argument('--pdf_json_path', type=str) # json format
# parser.add_argument('--pdf_latex_path', type=str) # latex format

# parser.add_argument('--output_dir',type=str, default="")

# args    = parser.parse_args()

In [ ]:
model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
tp_size = 2
temperature = 1.0
max_model_len = 128000

paper_format = "json"
pdf_json_path = None
pdf_latex_path = None

In [ ]:
caption_msg = [{'role': "system", "content": f"""You are an expert researcher and strategic planner with a deep understanding of experimental design and reproducibility in scientific research. 
You will receive a data pair in json format. 
Your task is find the figures from the data pair with the caption by using the caption alone, and return the figure path and the caption.

Instructions:

1. Using the textual caption, find the figure from the data pair that are in one of the three 
2. Be Clear and Structured: Present the plan in a well-organized and easy-to-follow format, breaking it down into actionable steps.
3. Prioritize Efficiency: Optimize the plan for clarity and practical implementation while ensuring fidelity to the original experiments."""}]


image_msg = [
        {'role': 'user', 'content': """You write elegant, modular, and maintainable code. Adhere to Google-style guidelines.

Based on the paper, plan, design specified previously, follow the "Format Example" and generate the code. 
Extract the training details from the above paper (e.g., learning rate, batch size, epochs, etc.), follow the "Format example" and generate the code. 
DO NOT FABRICATE DETAILS — only use what the paper provides.

You must write `config.yaml`.

ATTENTION: Use '##' to SPLIT SECTIONS, not '#'. Your output format must follow the example below exactly.

-----

# Format Example
## Code: config.yaml
```yaml
## config.yaml
training:
  learning_rate: ...
  batch_size: ...
  epochs: ...
...
```

-----

## Code: config.yaml
"""
    }]

In [ ]:
model_name = args.model_name
tokenizer = AutoTokenizer.from_pretrained(model_name)


if "Qwen" in model_name:
    llm = LLM(model=model_name, 
            tensor_parallel_size=tp_size, 
            max_model_len=max_model_len,
            gpu_memory_utilization=0.95,
            trust_remote_code=True, enforce_eager=True, 
            rope_scaling={"factor": 4.0, "original_max_position_embeddings": 32768, "type": "yarn"})
    sampling_params = SamplingParams(temperature=temperature, max_tokens=131072)


In [ ]:
def run_llm(msg):
    # vllm
    prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True) for messages in [msg]]

    outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)

    completion = [output.outputs[0].text for output in outputs]
    
    return completion[0] 

In [ ]:
responses = []
trajectories = []
total_accumulated_cost = 0
output_dir = "./image_agent_output"

In [ ]:
for idx, instruction_msg in enumerate([caption_msg, image_msg]):
    current_stage = ""
    if idx == 0 :
        current_stage = f"[image_agent] Caption extraction"
    elif idx == 1:
        current_stage = f"[image_agent] extract "
    print(current_stage)

    trajectories.extend(instruction_msg)

    completion = run_llm(trajectories)
    
    # response
    completion_json = {
        'text': completion
    }

    # print and logging
    print_response(completion_json, is_llm=True)

    responses.append(completion_json)

    # trajectories
    trajectories.append({'role': 'assistant', 'content': completion})


# save
os.makedirs(output_dir, exist_ok=True)

with open(f'{output_dir}/planning_response.json', 'w') as f:
    json.dump(responses, f)

with open(f'{output_dir}/planning_trajectories.json', 'w') as f:
    json.dump(trajectories, f)
